In [1]:
import pandas as pd
import requests
import json

In [2]:
query = """
https://data.austintexas.gov/resource/tyfh-5r8s.json?$select=checkout_kiosk_id%20as%20origin,%20return_kiosk_id%20as%20dest,%20count(trip_id)%20as%20count,%20checkout_date%20as%20time,%20date_extract_y(checkout_date)%20as%20year&$where=year%20%3E%202013&$group=origin,%20dest,%20time&$order=time
"""

In [3]:
chunk_size = 10000

offset = 0
res = requests.get(query + f"&$limit={chunk_size}&$offset={offset}")
data = json.loads(res.text)
output = []

while data:
    output = output + data
    offset += chunk_size
    res = requests.get(query + f"&$limit={chunk_size}&$offset={offset}")
    data = json.loads(res.text)

In [4]:
df = pd.DataFrame(output)
df["time"] = pd.to_datetime(df["time"]).dt.strftime("%Y-%m-%d")

In [5]:
years = df["year"].unique()

In [6]:
years

array(['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023'], dtype=object)

In [7]:
# reading an historical archive of metrobike kiosk locations
locs = pd.read_csv("locations_archive.csv")

In [9]:
# read in current dataset of locations
res = requests.get("https://data.austintexas.gov/resource/qd73-bsdg.json?$select=kiosk_id as id, kiosk_name as name, point_latitude(location) as lat, point_longitude(location) as lon")
data = json.loads(res.text)
odp = pd.DataFrame(data)

In [10]:
odp

,id,name,lat,lon
0,11,22nd 1/2 & Rio Grande,30.2862,-97.74516
1,111,23rd & San Gabriel,30.2874,-97.7478
2,1001,OFFICE/Main/Shop/Repair,30.27186,-97.73997
3,1002,6th & Navasota St.,30.26383,-97.72864
4,1003,8th & Guadalupe,30.27106,-97.74563
...,...,...,...,...
97,4061,Lakeshore @ Austin Hostel,30.24472,-97.72336
98,4062,Lakeshore & Pleasant Valley,30.24258,-97.71726
99,4699,East 5th/Shady @ Eastside Bus Plaza,30.25212,-97.69807
100,4879,16th/San Antonio,30.27924,-97.74371


In [32]:
a = pd.concat([odp,locs])
a["id"] = a["id"].astype(str)
a = a.drop_duplicates(subset=["id"], keep="first")

In [40]:
df = df[df["origin"].isin(list(a["id"]))]
df = df[df["dest"].isin(list(a["id"]))]

In [41]:
output_cols = ["origin", "dest", "count", "time"]
with pd.ExcelWriter("metrobike.xlsx") as writer:  
    for y in years:
        output_df = df[df["year"] == y]
        output_df[output_cols].to_excel(writer, sheet_name=f"{y} Trips", index=False)
    a.to_excel(writer, sheet_name=f"locations", index=False)